In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import confusion_matrix

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Merge, Dropout
from keras.layers import concatenate, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model, Sequential
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.initializers import glorot_uniform, he_uniform
from keras.optimizers import RMSprop, Adam
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

np.random.seed = 0

train_data_dir = 'image_classifier/training_data'
validation_data_dir = 'image_classifier/validation_data'
test_data_dir = 'image_classifier/test_data'

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   fill_mode = 'nearest',
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen   = ImageDataGenerator(preprocessing_function = preprocess_input)

test_datagen  = ImageDataGenerator(preprocessing_function = preprocess_input)

training_generator = train_datagen.flow_from_directory(train_data_dir,
                                                       target_size = (224, 224),
                                                       batch_size = batch_size,
                                                       class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(validation_data_dir,
                                                       target_size = (224, 224),
                                                       batch_size = batch_size,
                                                       class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size = (224, 224),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

In [ ]:
VGG16 = VGG16(weights = 'imagenet')

In [ ]:
CNN = VGG16.get_layer('fc2').output
CNN = Dense(8, activation = 'softmax', name = 'softmax', kernel_initializer = glorot_uniform())(CNN)

model = Model(inputs = VGG16.input, outputs = CNN)

In [ ]:
model.summary()

In [ ]:
# step 1: transfer learning
for layer in model.layers[:22]:
    layer.trainable = False
for layer in model.layers[22:]:
    layer.trainable = True

In [ ]:
# let's use default values of the Adam optimizer for transfer learning
adam_transfer = Adam(lr = 1e-4, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)
#sgd_transfer = SGD(lr = 0.1, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = adam_transfer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
class_weighting = {0: 5.6900,
                   1: 1.9965,
                   2: 1.5850,
                   3: 2.6343,
                   4: 3.2330,
                   5: 1.6119,
                   6: 0.6040,
                   7: 1.0000}

In [ ]:
# 5 epochs

history_transfer = model.fit_generator(generator = training_generator,
                                       steps_per_epoch = 3000 // batch_size,
                                       validation_data = validation_generator,
                                       validation_steps = 1000 // batch_size,
                                       epochs = 5,
                                       class_weight = class_weighting)

In [ ]:
# # step 2: finetuning
# for layer in model.layers[:4]:
#     layer.trainable = False
# for layer in model.layers[4:]:
#     layer.trainable = True

In [ ]:
# step 2: finetuning
for layer in model.layers:
    layer.trainable = True

In [ ]:
# let's use a smaller learning rate for finetuning since the weights are expected to be quite good already 
adam_finetune = Adam(lr = 1e-6, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)
model.compile(optimizer = adam_finetune, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# 15 epochs

history_finetune = model.fit_generator(generator = training_generator,
                                       steps_per_epoch = 3000 // batch_size,
                                       validation_data = validation_generator,
                                       validation_steps = 1000 // batch_size,
                                       epochs = 15,
                                       class_weight = class_weighting)

In [ ]:
# get the loss and accuracy of the test set
scores = model.evaluate_generator(generator = test_generator, steps = 1000 // batch_size)
print('Loss:', "{0:.3f}".format(scores[0]))
print('Test accuracy:', "{0:.2f}%".format(scores[1] * 100))

In [ ]:
# save the model
model.save('image_classifier_tl_5_ft_15.h5')

In [ ]:
# load the model
model = load_model('image_classifier_tl_5_ft_15.h5')

In [ ]:
# save the history files

picklehistory = open('history_transfer.p', 'wb')
pickle.dump(history_transfer.history, picklehistory, -1)
picklehistory.close()

picklehistory = open('history_finetune.p', 'wb')
pickle.dump(history_finetune.history, picklehistory, -1)
picklehistory.close()

In [ ]:
# import the history files

picklehistory = open('history_transfer.p', 'rb')
history_transfer.history = pickle.load(picklehistory)
picklehistory.close()

picklehistory = open('history_finetune.p', 'rb')
history_finetune.history = pickle.load(picklehistory)
picklehistory.close()

In [ ]:
test_set = list(zip(test_generator.filenames, test_generator.classes.tolist()))

In [ ]:
test_set_adj = []

for i in range(0, len(test_set)):
    img_path = 'image_classifier/test_data/' + test_set[i][0]
    img = image.load_img(img_path, target_size = (224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    
    probs_i = model.predict(x)
    pred_i = np.argmax(probs_i)
    
    test_i = [img_path] + [test_set[i][1]] + probs_i[0].tolist() + [pred_i]
    test_set_adj.append(test_i)

test_set_df = pd.DataFrame(test_set_adj)
test_set_df.columns = ['filename',
                       'y_true',
                       'prob_0',
                       'prob_1',
                       'prob_2',
                       'prob_3',
                       'prob_4',
                       'prob_5',
                       'prob_6',
                       'prob_7',
                       'y_hat']

In [ ]:
test_set_df['y_hat'].value_counts()

In [ ]:
confusion_matrix(test_set_df['y_true'], test_set_df['y_hat'])

In [ ]:
test_set_df.to_csv('test_set_predictions.csv')

In [ ]:
# predict for all the pictures

account_data_01 = pd.read_csv('results/dataset_analysis.csv', low_memory = False)

image_ids_train = pd.read_csv('results/image_ids_train.csv', low_memory = False)
image_ids_val = pd.read_csv('results/image_ids_val.csv', low_memory = False)
image_ids_test = pd.read_csv('results/image_ids_test.csv', low_memory = False)

In [ ]:
def file_path_from_train_df(image_id, label):
    image_path = 'training_data/' + str(label) + '/' + str(image_id) + '_256.jpg'
    return image_path

def file_path_from_validation_df(image_id, label):
    image_path = 'validation_data/' + str(label) + '/' + str(image_id) + '_256.jpg'
    return image_path

def file_path_from_test_df(image_id, label):
    image_path = 'test_data/' + str(label) + '/' + str(image_id) + '_256.jpg'
    return image_path

In [ ]:
train_df = account_data_01.merge(image_ids_train, on = 'image_id', how = 'inner')
train_df = train_df[['image_id', 'likes_groups']]
train_df['label'] = train_df.likes_groups.apply(lambda x: str(x)[0])
train_df['path'] = train_df.apply(lambda x: file_path_from_train_df(x['image_id'], x['label']), axis = 1)

validation_df = account_data_01.merge(image_ids_val, on = 'image_id', how = 'inner')
validation_df = validation_df[['image_id', 'likes_groups']]
validation_df['label'] = validation_df.likes_groups.apply(lambda x: str(x)[0])
validation_df['path'] = validation_df.apply(lambda x: file_path_from_validation_df(x['image_id'], x['label']), axis = 1)

test_df = account_data_01.merge(image_ids_test, on = 'image_id', how = 'inner')
test_df = test_df[['image_id', 'likes_groups']]
test_df['label'] = test_df.likes_groups.apply(lambda x: str(x)[0])
test_df['path'] = test_df.apply(lambda x: file_path_from_test_df(x['image_id'], x['label']), axis = 1)

In [ ]:
train_paths = list(zip(train_df['image_id'], train_df['path']))
validation_paths = list(zip(validation_df['image_id'], validation_df['path']))
test_paths = list(zip(test_df['image_id'], test_df['path']))

paths = train_paths + validation_paths + test_paths

In [ ]:
total = []

for i in range(0, len(paths)):
    img_path = paths[i][1]
    img = image.load_img(img_path, target_size = (224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)           
    
    probs_i = model.predict(x)
    pred_i = np.argmax(probs_i)
    
    record = [paths[i][0]] + [img_path] + probs_i[0].tolist() + [pred_i]
    total.append(record)
    
    if (i > 0) & (i % 5000 == 0):
        print(i, 'pictures processed')

total_df = pd.DataFrame(total)
total_df.columns = ['image_id',
                    'filename',
                    'prob_0',
                    'prob_1',
                    'prob_2',
                    'prob_3',
                    'prob_4',
                    'prob_5',
                    'prob_6',
                    'prob_7',
                    'y_hat']

In [ ]:
total_df.shape

In [ ]:
total_df.to_csv('total_predictions.csv')